In [6]:
#-- load the environment

from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
import codecs

import psycopg2
import pandas as pd

import tensorflow as tf
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import time
import pickle
import re


import nltk
from nltk.stem import PorterStemmer

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve

In [7]:
#-- connect to the postgresql sever

try:
    conn = psycopg2.connect(dbname = "postgres", user="readonly", host= "47.103.73.205" ,password= "123456", port  = "25432")
    print ("connection successful")
    cur = conn.cursor()
except:
    print("I am unable to connect to the database")
    
#-- selct the 10 items and then to pd.dataframe 
cur.execute("SELECT * FROM  meta_training_data_jd WHERE fun1_code LIKE 'ALG' or fun1_code LIKE 'BSC' ;")
rows=cur.fetchall()
df = pd.DataFrame(rows)




connection successful


In [8]:
df

,0,1,2,3,4,5,6,7,8,9,10
0,DIS,ALG004,ALG,004,企业/公司秘书（担负法务职责）,Corporate/Company Secretary (with Legal Respon...,203341,AppleOne,Legal Secretary -,\n\nJob Description:This Legal Secretary Posi...,monster
1,DIS,ALG004,ALG,004,企业/公司秘书（担负法务职责）,Corporate/Company Secretary (with Legal Respon...,212660,Staffing Now,Litigation Secretary,\n\nSNI Companies has partnered with one of th...,monster
2,DIS,ALG004,ALG,004,企业/公司秘书（担负法务职责）,Corporate/Company Secretary (with Legal Respon...,267508,"Beacon Hill Staffing Group, LLC",Legal Secretary,\n\nWe are currently in need of Legal Secretar...,monster
3,DIS,ALG004,ALG,004,企业/公司秘书（担负法务职责）,Corporate/Company Secretary (with Legal Respon...,267534,People People,Litigation Secretary,"\n\nMcAnany, Van Cleave and Phillips (MVP) is ...",monster
4,DIS,ALG004,ALG,004,企业/公司秘书（担负法务职责）,Corporate/Company Secretary (with Legal Respon...,267672,Kelly Services,Legal Secretary / Legal Assistant,\n\nJob DescriptionApply Now!Open Legal Secret...,monster
...,...,...,...,...,...,...,...,...,...,...,...
2424,DIS,ALG040,ALG,040,诉讼,Litigation,457667,"Cohen Compagni Beckman Appler & Knoll, PLLC",Legal Assistant - Litigation,\n\nJob Purpose:Serves clients by managing cas...,monster
2425,DIS,ALG004,ALG,004,企业/公司秘书（担负法务职责）,Corporate/Company Secretary (with Legal Respon...,176944,Legal Network Inc,Legal Secretary,\n\nTERRIFIC LEGAL SECRETARY POSITION AVAILABL...,monster
2426,DIS,ALG004,ALG,004,企业/公司秘书（担负法务职责）,Corporate/Company Secretary (with Legal Respon...,179380,Montana State Government,Administrative Support-Legal Secretary-Part-Time,\n\nDescription:To be considered for OPD posit...,monster
2427,DIS,ALG004,ALG,004,企业/公司秘书（担负法务职责）,Corporate/Company Secretary (with Legal Respon...,201104,Cummings & Lockwood LLC,Legal Secretary,\n\nLegal SecretaryCummings & Lockwood LLC Sta...,monster


In [9]:
stopwords = codecs.open("./stop_words.txt").read().splitlines()
stopwords

["'d",
 "'ll",
 "'m",
 "'re",
 "'s",
 "'t",
 "'ve",
 'ZT',
 'ZZ',
 'a',
 "a's",
 'able',
 'about',
 'above',
 'abst',
 'accordance',
 'according',
 'accordingly',
 'across',
 'act',
 'actually',
 'added',
 'adj',
 'adopted',
 'affected',
 'affecting',
 'affects',
 'after',
 'afterwards',
 'again',
 'against',
 'ah',
 "ain't",
 'all',
 'allow',
 'allows',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'an',
 'and',
 'announce',
 'another',
 'any',
 'anybody',
 'anyhow',
 'anymore',
 'anyone',
 'anything',
 'anyway',
 'anyways',
 'anywhere',
 'apart',
 'apparently',
 'appear',
 'appreciate',
 'appropriate',
 'approximately',
 'are',
 'area',
 'areas',
 'aren',
 "aren't",
 'arent',
 'arise',
 'around',
 'as',
 'aside',
 'ask',
 'asked',
 'asking',
 'asks',
 'associated',
 'at',
 'auth',
 'available',
 'away',
 'awfully',
 'b',
 'back',
 'backed',
 'backing',
 'backs',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming

Stemming is the process of reducing inflection in words (e.g. troubled, troubles) to their root form (e.g. trouble). The “root” in this case may not be a real root word, but just a canonical form of the original word.

Stemming uses a crude heuristic process that chops off the ends of words in the hope of correctly transforming words into its root form. So the words “trouble”, “troubled” and “troubles” might actually be converted to troubl instead of trouble because the ends were just chopped off (ughh, how crude!).

There are different algorithms for stemming. The most common algorithm, which is also known to be empirically effective for English, is Porters Algorithm. Here is an example of stemming in action with Porter Stemmer:

In [10]:
punctuation = '-!,;:?.-@()+&·*–…$"\''
porter_stemmer=PorterStemmer()
def removePunctuation(text):
    text = re.sub(r'[{}]+'.format(punctuation),'',text)
    text =  text.strip().lower()


    words=re.split("\\s+",text)
    stemmed_words=[porter_stemmer.stem(word=word) for word in words]
    return ' '.join(stemmed_words)
 
text = df.iloc[0,-2]
# print(removePunctuation(text))


In [11]:
docs = []
for row in rows:
    docs.append(removePunctuation(row[-2]))
docs[0]

'job descriptionthi legal secretari posit featuresawesom opportun for experienc legal secretari with year of experi to join small fun famili orient firm wear mani hat from order offic suppli type contract and legal document prepar present keep attorney schedul interact with client qualifi candid should have year of legal experi with strong type skill attent to detail strong work knowledg of ms offic fun outlook on life and daili work compani valu worklif balanc not in center citi free park on site we are an equal employ opportun employ commit to excel divers and inclus you can view all of our job onlin at httpwwwappleonecomscid'

In [12]:
id = []
for row in rows:
    id.append(row[1])

In [13]:
t0 = time.time()

count = TfidfVectorizer(max_df=0.5, min_df=2,  stop_words=stopwords)
transform=TfidfTransformer()
X=count.fit_transform(docs)
X=transform.fit_transform(X)
print(X.shape)
print()
feature=count.get_feature_names()


print("feature extraction done in %fs" % (time.time() - t0))
print()

### 将obj对象序列化存入已经打开的file中
f1=open('filter_data.p','wb')
pickle.dump(X,f1)
f2=open('feature_5000.p','wb')
pickle.dump(feature,f2)



C:\Users\Carl\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'al', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(2429, 15830)

feature extraction done in 0.907478s



In [14]:
def generate_feature_matrix(text, label):
    """Input: ('text', label: list or array-like), generate a feature matrix with word count and label)"""
    cv = CountVectorizer(analyzer='word', max_df=0.5, min_df=2,stop_words= stopwords)
    cv_fit = cv.fit_transform(text)

    feature_matrix = pd.DataFrame(cv_fit.toarray(), columns=cv.get_feature_names())
    feature_matrix.insert(0, "data_label", label)

    return feature_matrix

In [15]:
feature_matrix = generate_feature_matrix(docs, id)
# feature_matrix


In [16]:
# feature_matrix.describe()

## RandomForestClassifier

In [17]:
#-- X and y

X = feature_matrix.drop(columns = 'data_label')
y = feature_matrix.data_label


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42,stratify=y)

In [18]:
rf = RandomForestClassifier(max_depth = 10, random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [19]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score,recall_score


y_pred = rf.predict(X_test)
acc = accuracy_score(y_test,y_pred)
balanced = balanced_accuracy_score(y_test,y_pred)
recall = recall_score(y_test,y_pred,average='micro')
specificity = recall_score(y_test,y_pred, pos_label = 0,average='micro')

print('Acc. {} Recall: {} Specificity: {} Balanced: {}'.format(acc,recall,specificity,balanced))

Acc. 0.5386533665835411 Recall: 0.5386533665835411 Specificity: 0.5386533665835411 Balanced: 0.3741470192512721


C:\Users\Carl\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1317: UserWarning: Note that pos_label (set to 0) is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


# XGBClassifier

In [20]:
param_dist = {'objective':'binary:logistic', 'n_estimators':2}

clf = XGBClassifier(**param_dist)

clf.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        eval_metric="merror",
        verbose=True,)

evals_result = clf.evals_result()

[0]	validation_0-merror:0.22803	validation_1-merror:0.36658
[1]	validation_0-merror:0.16964	validation_1-merror:0.34040


 # Evaluation

In [21]:
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test,y_pred)
balanced = balanced_accuracy_score(y_test,y_pred)
recall = recall_score(y_test,y_pred,average='micro')
specificity = recall_score(y_test,y_pred, pos_label = 0,average='micro')

print('Acc. {} Recall: {} Specificity: {} Balanced: {}'.format(acc,recall,specificity,balanced))

Acc. 0.6596009975062345 Recall: 0.6596009975062345 Specificity: 0.6596009975062345 Balanced: 0.5760635093064937


--------------------

In [22]:
import sys 
import time
import random
import numpy as np
import os 
import pickle
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, MaxPool1D, GlobalMaxPooling1D
from keras.preprocessing import sequence


In [25]:
## define hyperparameters 
## Using TensorFlow backend.

max_word=500 
max_loop = 1000 
vocab_size=4000 
batch_size = 32 
category_size=15830


## prepare the data 
##with open('./filter_data.p', 'rb') as f: 
#    doc_vector=pickle.load(f)
    
##with open('./feature_5000.p','rb') as f: 
#    label=pickle.load(f)

doc_vector = X
label = feature

aux=np.eye(category_size) 
label_onehot=np.array([aux[y] for y in label])

random.seed = 100 
train_proportion=0.7 
max_length=1497

state = np.random.get_state()
np.random.shuffle(doc_vector)
np.random.set_state(state) 
np.random.shuffle(label_onehot)


total_size = len(label) 
cut = int(np.ceil(total_size * train_proportion))
x_train = doc_vector[:cut] 
x_train=sequence.pad_sequences(x_train,maxlen=max_length) 
y_train = label_onehot[:cut]
x_test = doc_vector[cut:] 
x_test=sequence.pad_sequences(x_test,maxlen=max_length) 
y_test = label_onehot[cut:]


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [27]:
doc_vector

,__,_busi,_close,_directorate_,_feb,_financeaccountingauditing_,_group_,_job,_mo,_open,...,zeh,zerohungerzerowast,zillow,zion,zip,zone,zr,zuora,zurich,zycu
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2424,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2425,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2426,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2427,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
with open('split_data.p','wb') as f: pickle.dump((x_train,y_train,x_test,y_test),f)

In [ ]:
 ## build the model 
model=Sequential() 
model.add(Embedding(vocab_size,64,input_length=max_length)) 
model.add(Dropout(0.2))
# we add a Convolution1D, which will learn filters 
# word group filters of size filter_length: 
model.add(Conv1D(250, 3, padding='valid', activation='relu', strides=1)) 
# we use max pooling: 
model.add(GlobalMaxPooling1D())
# We add a vanilla hidden layer: 
model.add(Dense(32)) 
model.add(Dropout(0.2)) model.add(Activation('relu'))
# We project onto a single unit output layer, and squash it with a sigmoid: 
model.add(Dense(category_size,activation='softmax')) 
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print('the model is loaded!')


In [ ]:
model.fit(x_train, y_train, validation_data=(x_test,y_test),epochs=20,batch_size=64) 
scores=model.evaluate(x_test,y_test,verbose=1) 
print(scores)

